## **Exploratory Data Analysis**

In this notebook we will explore the different datasets and visualize them

Importing Libraries

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_SUMMARY.csv',delimiter=';', encoding='unicode_escape',usecols=["Latitude","Longitude","Road_type","Event/action_type"])

# headway_warning=[41]
df = df[df['Event/action_type']==41]
# df = df[df['Road_type']==0]
with ProgressBar():
    df = df.compute()

# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\brake_warnings.csv")

In [ ]:
%%capture
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import pathlib
from pathlib import *
import os
import pickle
import seaborn as sns
import plotly.express as px
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import hvplot.pandas
import holoviews as hv, pandas as pd, colorcet as cc
from holoviews.element.tiles import EsriImagery
from holoviews.operation.datashader import datashade
hv.extension('bokeh')
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import os
import datashader
import datashader.transfer_functions
from datashader.utils import lnglat_to_meters
from bokeh.models import BoxZoomTool
from bokeh.plotting import figure, output_notebook, show
from datashader.utils import lnglat_to_meters
from bokeh.tile_providers import get_provider, Vendors 
import holoviews.operation.datashader as hd
from datashader.colors import Hot
import datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import Greys9

output_notebook()

### Bokeh Plots

In [ ]:
## Class for viz aos data points -- Bokeh Plots 
class Map_Viz:
    
    # Define general parameters for the Bokeh plot
    Netherlands = x_range, y_range = ((384019.630105,865878.656414), (6577253.409883,7156951.832398))

    plot_width  = int(990)
    plot_height = int(plot_width//1.2)

    
    tile_provider = get_provider(Vendors.CARTODBPOSITRON)
    
    def __init__(self,event_file_name):
        self.event_file_name=event_file_name
        
    def import_data_aos(self):
        """
        Load data from a csv file.
        """
        fields = ['Latitude', 'Longitude']
        complete_path=os.path.join(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY", self.event_file_name)
        df=pd.read_csv(complete_path,delimiter=';', skipinitialspace=True, usecols=fields, encoding= 'unicode_escape',dtype={'Latitude': float,'Longitude':float},skiprows=range(1,2))
        return df

    # Reusable function to create simple Bokeh plots

    def base_plot(self,tools='pan,wheel_zoom,reset', plot_width=plot_width, plot_height=plot_height, **plot_kwargs):
        base_plot = figure(tools=tools, plot_width=Map_Viz.plot_width, plot_height=Map_Viz.plot_height,
            x_range=Map_Viz.x_range, y_range=Map_Viz.y_range, outline_line_color=None,
            min_border=0, min_border_left=0, min_border_right=0,
            min_border_top=0, min_border_bottom=0, **plot_kwargs)
        
        base_plot.axis.visible = False
        base_plot.xgrid.grid_line_color = None
        base_plot.ygrid.grid_line_color = None
        
        base_plot.add_tools(BoxZoomTool(match_aspect=True))
        
        return base_plot
    
    def process_data(self):
        df=self.import_data_aos()
        df["Longitude"] = df["Longitude"].astype(float)
        df["Latitude"] = df["Latitude"].astype(float)
        
        df.loc[:, 'Longitude'], df.loc[:, 'Latitude'] = lnglat_to_meters(df.Longitude,df.Latitude)
        
        return df       
    
    def plot_map(self,number_of_data_points):
        df=self.process_data()
        samples = df.sample(n=number_of_data_points)
        p = self.base_plot()
        p.add_tile(Map_Viz.tile_provider)
        options = dict(line_color=None, fill_color='forestgreen', size=5)
        p.circle(x=samples['Longitude'], y=samples['Latitude'], **options)
        show(p)
    
    def plot_map_better(self,number_of_data_points):
        df=self.process_data()
        samples = df.sample(n=number_of_data_points)
        options = dict(line_color=None, fill_color='forestgreen', size=1, alpha=0.1)
        p = self.base_plot()
        p.add_tile(Map_Viz.tile_provider)
        p.circle(x=samples['Longitude'], y=samples['Latitude'], **options)
        show(p)
        



map_viz = Map_Viz("AOS_SUMMARY.csv")

"""Plotting individual events -- uncomment below"""
# map_viz=Map_Viz("brake_warnings.csv") 
# map_viz=Map_Viz("headway_warnings.csv")
# map_viz=Map_Viz("right_left_lane_departure_warning.csv")

# map_viz.plot_map(number_of_data_points=1000000) # overplotting
map_viz.plot_map_better(number_of_data_points=1000000) # no overplotting

### Plot all data points

In [ ]:
## Class for viz all data points
"""https://datashader.org/ --> Big data viz library """

class Map_Viz_Datashader:
    plot_width  = int(990)
    plot_height = int(plot_width//1.2)
    Netherlands = x_range, y_range = ((384019.630105,865878.656414), (6577253.409883,7156951.832398))
    def __init__(self,event_file_name):
        self.event_file_name=event_file_name
        
        
    def import_data_td(self):
        """
        Load data from a csv file.
        """

        fields = ['Latitude', 'Longitude']
        complete_path=os.path.join(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL", self.event_file_name)
        df=pd.read_csv(complete_path,delimiter=';', skipinitialspace=True, usecols=fields, encoding= 'unicode_escape',dtype={'Latitude': float,'Longitude':float},skiprows=range(1,2))
        return df
        
    def import_data_aos(self):
            fields = ['Latitude', 'Longitude']
            complete_path=os.path.join(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY", self.event_file_name)
            df=pd.read_csv(complete_path,delimiter=';', skipinitialspace=True, usecols=fields, encoding= 'unicode_escape',dtype={'Latitude': float,'Longitude':float},skiprows=range(1,2))
        
            return df
        
    def process_data_aos(self):
        df=self.import_data_aos()
        df["Longitude"] = df["Longitude"].astype(float)
        df["Latitude"] = df["Latitude"].astype(float)
        
        df.loc[:, 'Longitude'], df.loc[:, 'Latitude'] = lnglat_to_meters(df.Longitude,df.Latitude)
        
        return df    
    
    def process_data_td(self):
        df=self.import_data_td()
        df["Longitude"] = df["Longitude"].astype(float)
        df["Latitude"] = df["Latitude"].astype(float)
        
        df.loc[:, 'Longitude'], df.loc[:, 'Latitude'] = lnglat_to_meters(df.Longitude,df.Latitude)
        
        return df    
    

"""Plot Trip Detail - uncomment below"""
# map_viz = Map_Viz_Datashader("TRIP_DETAIL.csv")
# df=map_viz.process_data_td() 

"""Plotting individual events -- uncomment below"""
# map_viz=Map_Viz_Datashader("brake_warnings.csv") 
# map_viz=Map_Viz_Datashader("headway_warnings.csv")
# map_viz=Map_Viz_Datashader("right_left_lane_departure_warning.csv")
map_viz=Map_Viz_Datashader("AOS_SUMMARY.csv")
df=map_viz.process_data_aos() 

plot_width  = int(990)
plot_height = int(plot_width//1.2)
Netherlands = x_range, y_range = ((384019.630105,865878.656414), (6577253.409883,7156951.832398))

cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height, x_range=x_range, y_range=y_range)
agg = cvs.points(df, 'Longitude', 'Latitude')
img = tf.shade(agg, cmap="green", how='log')
img

In [ ]:
## Save image locally
# bgcolor can be white/black/whatever
from datashader.utils import export_image
export_image(img, "out1", background="white", export_path=".")

In [ ]:
##TODO -> Create functions for repetitive tasks
# ,nrows=20000000,skiprows=range(3, 260000000)
# ,skiprows=range(3, 260000000)
# Input the csv
# Extracting a subset of 1000000 rows by default
def load_data(dir_name,base_filename):
    complete_path=os.path.join(dir_name, base_filename + "." + "csv")
    # df=pd.read_csv(complete_path,sep=';',encoding= 'unicode_escape',nrows=10000,engine='c',infer_datetime_format=True,usecols=['Trip_Summary_Id','Numberplate','Point_time-stamp','Latitude',
    #                                                                                                                           'Longitude','Meters_travelled','Time_elapsed','Point_speed','Number_of_lanes',
    #                                                                                                                           'Road_type','Road_class','Road_form','Speed_restriction','TNO_Time-stamp'])
    # df = pd.concat((chunk for chunk in pd.read_csv(complete_path,sep=';',nrows=5000000,encoding= 'unicode_escape',engine='c',infer_datetime_format=True,chunksize=1000000,low_memory=False,usecols=['Numberplate','Latitude',
    #                                                                                                                           'Longitude','Meters_travelled','Time_elapsed','Point_speed','Number_of_lanes',
    #                                                                                                                           'Road_type','Road_class','Speed_restriction','TNO_Time-stamp'])))
    
    df = pd.concat((chunk for chunk in pd.read_csv(complete_path,sep=';',encoding= 'unicode_escape',engine='c',infer_datetime_format=True,chunksize=1000000,low_memory=False)))
    return df

# Dropping the first and last row of csv ("------")
def drop_first_row(df):
    df=df.iloc[1:]
    df=df[:-1]
    return df

def resetIndex(df):
    return df.reset_index()


##TODO -> Rewrite this function

def cast_to_correct_dtype(df):
    
    if 'Id' in df.columns:
        df['Id'] = df['Id'].astype('int')
        
    if 'AOS_position_Id' in df.columns:
        df['AOS_position_Id'] = df['AOS_position_Id'].astype('int')
    
    if 'Acceleration_x' in df.columns:
        df['Acceleration_x'] = df['Acceleration_x'].astype('float')
        
    if 'Acceleration_y' in df.columns:
        df['Acceleration_y'] = df['Acceleration_y'].astype('float')
        
    if 'TNO_Valid' in df.columns:
        df['TNO_Valid'] = df['TNO_Valid'].astype('int')
    
    if 'Latitude' in df.columns:
        df['Latitude'] = df['Latitude'].astype('float')
        
    if 'Longitude' in df.columns:
        df['Longitude'] = df['Longitude'].astype('float')
        
    if 'Event/action_speed' in df.columns:
        df['Event/action_speed'] = df['Event/action_speed'].astype('int')
        
    if 'Event/action_type' in df.columns:
        df['Event/action_type'] = df['Event/action_type'].astype('int')    
        
    if 'Number_of_lanes' in df.columns:
        df['Number_of_lanes'] = df['Number_of_lanes'].astype('int')
        
    if 'Road_class' in df.columns:
        df['Road_class'] = df['Road_class'].astype('int')
        
    if 'Road_type' in df.columns:
        df['Road_type'] = df['Road_type'].astype('int')
        
    if 'Crash_position_Id' in df.columns:
        df['Crash_position_Id']=df['Crash_position_Id'].astype('int')
        
    if 'Crash_position_Id' in df.columns:
        df['Crash_position_Id'] = df['Crash_position_Id'].astype('int')
        
    if 'Point_speed' in df.columns:
        df['Point_speed'] = df['Point_speed'].astype('float')
        
    if 'Average_speed_fpp' in df.columns:
        df['Average_speed_fpp'] = df['Average_speed_fpp'].astype('float')  
        
    if 'Average_Speed' in df.columns:
        df['Average_Speed'] = df['Average_Speed'].astype('float')        

    if 'Maximum_speed' in df.columns:
        df['Maximum_speed'] = df['Maximum_speed'].astype('float')     
        
    if 'Meters_travelled' in df.columns:
        df['Meters_travelled'] = df['Meters_travelled'].astype('int')  

    if 'Road_form' in df.columns:
        df['Road_form'] = df['Road_form'].astype('int') 
        
    if 'Speed_restriction' in df.columns:
        df['Speed_restriction'] = df['Speed_restriction'].astype('int') 
        
    if 'Crash_speed' in df.columns:
        df['Crash_speed'] = df['Crash_speed'].astype('int')
        
    if 'Maximum_acceleration' in df.columns:
        df['Maximum_acceleration'] = df['Maximum_acceleration'].astype('float')
        
    if 'Numberplate' in df.columns:
        df['Numberplate']=df['Numberplate'].astype('str')
        

    return df

            

## Handling date time related fields

def cast_date_time(df):
    if 'TNO_Trip-start' in df.columns:
        df['TNO_Trip-start'] = pd.to_datetime(df['TNO_Trip-start'],errors='coerce')
        df['TNO_Trip-start_hour'] = df['TNO_Trip-start'].dt.hour
        
    if 'TNO_Trip-end' in df.columns:
        df['TNO_Trip-end'] = pd.to_datetime(df['TNO_Trip-end'],errors='coerce')
        
    if 'Position_time' in df.columns:
        df['Position_time'] = pd.to_datetime(df['Position_time'],errors='coerce')

    if 'TNO_Time-stamp' in df.columns:
        df['TNO_Time-stamp'] = pd.to_datetime(df['TNO_Time-stamp'],errors='coerce')
        df['TNO_Time-stamp_hour'] = df['TNO_Time-stamp'].dt.hour
        
    if 'Trip_start_time-stamp' in df.columns:
        df['Trip_start_time-stamp'] = pd.to_datetime(df['Trip_start_time-stamp'],errors='coerce')
        df['Trip_start_time-stamp_hour'] = df['Trip_start_time-stamp'].dt.hour
        
    if 'Trip_start_date-stamp' in df.columns:
        df['Trip_start_date-stamp'] = pd.to_datetime(df['Trip_start_date-stamp'],errors='coerce')
        df['Trip_start_date-stamp_hour'] = df['Trip_start_date-stamp'].dt.hour
    return df

## Converting Time Stamps to datetime

def date_and_time_columns(df):
    if 'TNO_Time-stamp' in df.columns:
        df['TNO Trip Date'] = df['TNO_Time-stamp'].dt.date
        df['TNO Trip Time'] = df['TNO_Time-stamp'].dt.time
        df['TNO Trip Hour']=df['TNO_Time-stamp'].dt.hour
        df['TNO Trip Month'] = df['TNO_Time-stamp'].dt.hour
    

    if 'Trip_start_date-stamp' in df.columns:
        df['TNO Trip Date'] = df['Trip_start_date-stamp'].dt.date
        # df['TNO Trip Time'] = df['Trip_start_date-stamp'].dt.time
        # df['TNO Trip Hour']=df['Trip_start_date-stamp'].dt.hour
        df['TNO Trip Month']=df['Trip_start_date-stamp'].dt.month
        
    if 'Trip_start_time-stamp' in df.columns:
        # df['TNO Trip Date'] = df['Trip_start_date-stamp'].dt.date
        df['TNO Trip Time'] = df['Trip_start_time-stamp'].dt.time
        df['TNO Trip Hour']=df['Trip_start_time-stamp'].dt.hour
        # df['TNO Trip Month']=df['Trip_start_date-stamp'].dt.month
        
    return df


def Time_of_the_day(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x < 12 ):
        return 'Morning'
    elif (x >= 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Evening'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'
    
    
def rename_some_stuff(df):
    if 'Event/action_type' in df.columns:
        df["Event type Rename"]=df["Event/action_type"].replace({0: 'Headway Warning = OFF', 1: 'Headway Warning (long)', 2: 'Headway Warning (medium)'
                                                                                 ,3: 'Headway Warning (short)',10:'Lane Departure Warning = OFF',11:'Left Lane Departure Warning = ON'
                                                                                 ,12:'Right Lane Departure Warning = ON',13:'Left and Right Lane Departure Warning = ON'
                                                                                 ,20:' Indicators = OFF',21:'Left Indicator = ON',22:'Right Indicator = ON',23:'Left and Right Indicator = ON'
                                                                                 ,40:'Brakes = OFF',41:'Brakes = ON'})
        
    if 'Road_type' in df.columns:
        df["Road_type_Rename"]=df["Road_type"].replace({0: "Urban", 1: "Motorway",2:"Extra Urban",3:"Unavailable"})
        
    # if 'TNO Trip Month' in df.columns:
    #     df["TNO_Trip_Month_Rename"]=df["TNO Trip Month"].replace({1: "January", 2: "February",3:"March",4:"April",5:"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"})
        
    return df

def detect_overspeeding_count(df):
    # create a list of our conditions
    # if 'Event/action_speed' and 'Speed_restriction' in df.columns:
    #     conditions = [(df['Event/action_speed']> df['Speed_restriction'])]
    #     # # create a list of the values we want to assign for each condition
    #     values = [1]
    #     # create a new column and use np.select to assign values to it using our lists as arguments
    #     df['Overspeeding_event'] = np.select(conditions, values)
        
    # return df

    if 'Point_speed' and 'Speed_restriction' in df.columns:
        conditions = [(df['Point_speed']> df['Speed_restriction'])]
        # # create a list of the values we want to assign for each condition
        values = [1]
        # create a new column and use np.select to assign values to it using our lists as arguments
        df['Overspeeding_event'] = np.select(conditions, values)
        
    return df

def some_processing(df):
    if 'Meters_travelled' in df.columns:
        df['KmTravelled']=df['Meters_travelled']/1000
    return df

In [ ]:
df_TRIP_SUMMARY = (load_data(r"D:\AOS FOT\Octo\CSV Export\TRIP_SUMMARY","TRIP_SUMMARY")
                                      .pipe(drop_first_row)
                                      .pipe(resetIndex)
                                      .pipe(cast_to_correct_dtype)
                                      .pipe(cast_date_time)
                                      .pipe(date_and_time_columns)
                                      .pipe(rename_some_stuff))
df_TRIP_SUMMARY['Part of Day'] = df_TRIP_SUMMARY['TNO Trip Hour'].apply(Time_of_the_day)
df_TRIP_SUMMARY['Count']=1
# df_TRIP_SUMMARY

In [ ]:
month_grp = df_TRIP_SUMMARY.groupby("Trip_start_date-stamp")["Meters_travelled"].sum()
month_grp=month_grp.to_frame()
month_grp=month_grp.reset_index()
month_grp['km_travelled']=month_grp['Meters_travelled']/1000
month_grp

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook

# Load a numpy structured array from yahoo csv data with fields date, open,
# close, volume, adj_close from the mpl-data/example directory.  This array
# stores the date as an np.datetime64 with a day unit ('D') in the 'date'
# column.
data = month_grp

fig, axs = plt.subplots(3, 1, figsize=(6.4, 7), constrained_layout=True)
# common to all three:
for ax in axs:
    ax.plot('Trip_start_date-stamp', 'Meters_travelled', data=data)
    # Major ticks every half year, minor ticks every month,
    ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 7)))
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.grid(True)
    ax.set_ylabel(r'Price [\$]')

# different formats:
ax = axs[0]
ax.set_title('DefaultFormatter', loc='left', y=0.85, x=0.02, fontsize='medium')

ax = axs[1]
ax.set_title('ConciseFormatter', loc='left', y=0.85, x=0.02, fontsize='medium')
ax.xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))

ax = axs[2]
ax.set_title('Manual DateFormatter', loc='left', y=0.85, x=0.02,
             fontsize='medium')
# Text in the x axis will be displayed in 'YYYY-mm' format.
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
# Rotates and right-aligns the x labels so they don't crowd each other.
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

plt.show()

In [ ]:
sns.barplot(x="Trip_start_date-stamp", y="Meters_travelled",data=month_grp)

In [ ]:
month_grp

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 7))

# sns.lineplot(x=month_grp["Trip_start_date-stamp"].dt.month, y=month_grp['Meters_travelled'], ax=ax)
# # ax.legend(title='Month', bbox_to_anchor=(1, 1), loc='upper left')
# ax.set(xlabel='Month')
# plt.show()

In [ ]:
px.line(month_grp, x = month_grp["Trip_start_date-stamp"], y = month_grp["Meters_travelled"]/1000,title="Total distance covered (Meters) over time",template="seaborn").update_layout(
    xaxis_title="Date", yaxis_title="Distance Travelled (Meters)"
)

In [ ]:
df_TRIP_SUMMARY

In [ ]:
merge_grp = df_TRIP_SUMMARY.groupby(["TNO Trip Month", "TNO Trip Hour"])["Meters_travelled"].sum()
merge_grp=merge_grp.to_frame()
merge_grp=merge_grp.reset_index()
merge_grp
# month_grp['km_travelled']=month_grp['Meters_travelled']/1000
# month_grp

In [ ]:
merge_grp["TNO_Trip_Month_Rename"]=merge_grp["TNO Trip Month"].replace({1: "January", 2: "February",3:"March",4:"April",5:"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"})

In [ ]:
sns.lineplot(x=merge_grp['TNO Trip Hour'],y=merge_grp['Meters_travelled'],hue=merge_grp["TNO_Trip_Month_Rename"])

In [ ]:
month_grp.plot()

In [ ]:
df_TRIP_SUMMARY.columns

In [ ]:

%%capture
import pandas as pd
# import modin.pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
# import vaex # https://vaex.io/docs/index.html
import pathlib
from pathlib import *
import os
import pickle
# import cufflinks as cf
# import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot

init_notebook_mode(connected=True)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [ ]:
df1_TD=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv")

In [ ]:
df1_TD['Month'] = pd.DatetimeIndex(df1_TD['TNO_Time-stamp']).month
df1_TD

In [ ]:
merge_grp = df1_TD.groupby(["Month", "TNO Trip Hour"])["Meters_travelled"].sum()
merge_grp=merge_grp.to_frame()
merge_grp=merge_grp.reset_index()
merge_grp

In [ ]:
merge_grp.to_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month_hour\month_hour_td40M.csv")

In [ ]:
merge_grp = df1_TD.groupby("Month")["Meters_travelled"].sum()
merge_grp=merge_grp.to_frame()
merge_grp=merge_grp.reset_index()
merge_grp

In [ ]:
merge_grp.to_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month40M.csv")

In [ ]:
merge_grp = df1_TD.groupby("TNO Trip Hour")["Meters_travelled"].sum()
merge_grp=merge_grp.to_frame()
merge_grp=merge_grp.reset_index()
merge_grp

In [ ]:
merge_grp.to_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\hour\hour40M.csv")

In [ ]:
merge_grp = df1_TD.groupby("TNO Trip Hour")["Point_speed"].mean()
merge_grp=merge_grp.to_frame()
merge_grp=merge_grp.reset_index()
merge_grp

In [ ]:
merge_grp.to_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\hour\hour_pointspeed40M.csv")

In [ ]:
sns.lineplot(x=merge_grp['TNO Trip Hour'],y=merge_grp['Meters_travelled'],hue=merge_grp["Month"])

In [ ]:
df1_TD.columns

In [ ]:
month20M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month20M.csv")
month20M.drop(['Unnamed: 0'], axis = 1, inplace = True)
month20M.rename(columns={'Meters_travelled': 'Meters_travelled20M'},inplace=True, errors='raise')


month40M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month40M.csv")
month40M.drop(['Unnamed: 0'], axis = 1, inplace = True) 
month40M.rename(columns={'Meters_travelled': 'Meters_travelled40M'},inplace=True, errors='raise')

month60M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month60M.csv")
month60M.drop(['Unnamed: 0'], axis = 1, inplace = True)
month60M.rename(columns={'Meters_travelled': 'Meters_travelled60M'},inplace=True, errors='raise')

month80M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month80M.csv")
month80M.drop(['Unnamed: 0'], axis = 1, inplace = True) 
month80M.rename(columns={'Meters_travelled': 'Meters_travelled80M'},inplace=True, errors='raise')


month100M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month100M.csv")
month100M.drop(['Unnamed: 0'], axis = 1, inplace = True) 
month100M.rename(columns={'Meters_travelled': 'Meters_travelled100M'},inplace=True, errors='raise')

month120M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month120M.csv")
month120M.drop(['Unnamed: 0'], axis = 1, inplace = True) 
month120M.rename(columns={'Meters_travelled': 'Meters_travelled120M'},inplace=True, errors='raise')

month140M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month140M.csv")
month140M.drop(['Unnamed: 0'], axis = 1, inplace = True) 
month140M.rename(columns={'Meters_travelled': 'Meters_travelled140M'},inplace=True, errors='raise')

month160M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month160M.csv")
month160M.drop(['Unnamed: 0'], axis = 1, inplace = True) 
month160M.rename(columns={'Meters_travelled': 'Meters_travelled160M'},inplace=True, errors='raise')

month180M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month180M.csv")
month180M.drop(['Unnamed: 0'], axis = 1, inplace = True) 
month180M.rename(columns={'Meters_travelled': 'Meters_travelled180M'},inplace=True, errors='raise')

month200M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month200M.csv")
month200M.drop(['Unnamed: 0'], axis = 1, inplace = True) 
month200M.rename(columns={'Meters_travelled': 'Meters_travelled200M'},inplace=True, errors='raise')

month220M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month220M.csv")
month220M.drop(['Unnamed: 0'], axis = 1, inplace = True) 
month220M.rename(columns={'Meters_travelled': 'Meters_travelled220M'},inplace=True, errors='raise')

month240M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month240M.csv")
month240M.drop(['Unnamed: 0'], axis = 1, inplace = True)
month240M.rename(columns={'Meters_travelled': 'Meters_travelled240M'},inplace=True, errors='raise') 
month260M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month260M.csv")
month260M.drop(['Unnamed: 0'], axis = 1, inplace = True) 
month260M.rename(columns={'Meters_travelled': 'Meters_travelled260M'},inplace=True, errors='raise')
month280M=pd.read_csv("D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\month\month280M.csv")
month280M.drop(['Unnamed: 0'], axis = 1, inplace = True) 
month280M.rename(columns={'Meters_travelled': 'Meters_travelled280M'},inplace=True, errors='raise')

In [ ]:
month20M

In [ ]:
import pandas as pd
from functools import reduce

# compile the list of dataframes you want to merge
data_frames = [month20M, month40M, month60M,month80M,month100M,month120M,month140M,month160M,month180M,month200M,month220M,month240M,month260M,month280M]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Month'],how='outer'), data_frames).fillna(0)
# df["TNO_Trip_Month_Rename"]=df["TNO Trip Month"].replace({1: "January", 2: "February",3:"March",4:"April",5:"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"})

In [ ]:
df_merged=df_merged.sort_values(by=['Month'],ascending=True)
df_merged

In [ ]:
list_of_my_columns = ['Meters_travelled20M', 'Meters_travelled40M', 'Meters_travelled60M','Meters_travelled80M','Meters_travelled100M','Meters_travelled120M','Meters_travelled140M','Meters_travelled160M','Meters_travelled180M','Meters_travelled200M','Meters_travelled220M','Meters_travelled240M','Meters_travelled260M','Meters_travelled280M']
df_merged['total_distance'] = df_merged[list_of_my_columns].sum(axis=1)

In [ ]:
df_merged

In [ ]:
df_merged["TNO_Trip_Month_Rename"]=df_merged["Month"].replace({1: "January", 2: "February",3:"March",4:"April",5:"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"})
df_merged

In [ ]:
Total = df_merged['total_distance'].sum()
print(Total/1000)

In [ ]:
sns.barplot(x=df_merged["TNO_Trip_Month_Rename"],y=df_merged["total_distance"])

In [ ]:

%%capture
import pandas as pd
# import modin.pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
# import vaex # https://vaex.io/docs/index.html
import pathlib
from pathlib import *
import os
import pickle
# import cufflinks as cf
# import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot

init_notebook_mode(connected=True)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [ ]:
df=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_140M.csv")

In [ ]:
len(df)

In [ ]:
df['Event type Rename'].value_counts()

In [ ]:
df=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df280M.csv")
# df['Count'] = 1
# df = df.drop(df[df['Point_speed']==0].index)
# df['TNO_Time-stamp'] = pd.to_datetime(df['TNO_Time-stamp'],errors='coerce')
# df['Month'] = pd.DatetimeIndex(df['TNO_Time-stamp']).month
# df['weekday'] = df['TNO_Time-stamp'].dt.dayofweek

In [ ]:
len(df)

In [ ]:
df_urban= df.loc[df['Road_type']==0] #Urban
df_motorway= df.loc[df['Road_type']==1] #Urban
df_extra_urban= df.loc[df['Road_type']==2] #Urban

In [ ]:
len(df_extra_urban)

#### Urban

In [ ]:
merge_grp_urban = df_urban.groupby(["TNO Trip Hour","weekday"]).sum()
# merge_grp_urban=merge_grp_urban.to_frame()
merge_grp_urban=merge_grp_urban.reset_index()
merge_grp_urban["weekday"]=merge_grp_urban["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})
# merge_grp_urban

In [ ]:
merge_grp_urban.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\sum_hour\urban\sum_hour1_urban.csv")

In [ ]:
merge_grp_motorway = df_motorway.groupby(["TNO Trip Hour","weekday"]).sum()
# merge_grp_motorway=merge_grp_motorway.to_frame()
merge_grp_motorway=merge_grp_motorway.reset_index()
merge_grp_motorway["weekday"]=merge_grp_motorway["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})
# merge_grp_motorway

In [ ]:
merge_grp_motorway.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\sum_hour\motorway\sum_hour1_motorway.csv")

In [ ]:
merge_grp_extra_urban = df_extra_urban.groupby(["TNO Trip Hour","weekday"]).sum()
# merge_grp_extra_urban=merge_grp_extra_urban.to_frame()
merge_grp_extra_urban=merge_grp_extra_urban.reset_index()
merge_grp_extra_urban["weekday"]=merge_grp_extra_urban["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})

In [ ]:
merge_grp_extra_urban.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\sum_hour\extra_urban\sum_hour1_extra_urban.csv")

In [ ]:
# merge_grp_urban = df_urban.groupby(["Month","TNO Trip Hour"]).sum()
# # merge_grp_urban=merge_grp_urban.to_frame()
# merge_grp_urban=merge_grp_urban.reset_index()
# merge_grp_urban["weekday"]=merge_grp_urban["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})
# merge_grp_urban

In [ ]:
# merge_grp_urban.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\sum_month\urban\sum_month13_urban.csv")

In [ ]:
# merge_grp_urban = df_urban.groupby(["Month"]).mean()
# # merge_grp_urban=merge_grp_urban.to_frame()
# merge_grp_urban=merge_grp_urban.reset_index()
# merge_grp_urban["weekday"]=merge_grp_urban["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})
# # merge_grp_urban

In [ ]:
# merge_grp_urban.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\mean_month\urban\mean_month13_urban.csv")

In [ ]:
# merge_grp_urban = df_urban.groupby(["TNO Trip Hour"]).mean()
# # merge_grp_urban=merge_grp_urban.to_frame()
# merge_grp_urban=merge_grp_urban.reset_index()
# merge_grp_urban["weekday"]=merge_grp_urban["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})
# merge_grp_urban

In [ ]:
# merge_grp_urban.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\mean_hour\urban\mean_hour13_urban.csv")

### Motorway

In [ ]:
# merge_grp_motorway.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\mean_month\motorway\mean_month13_motorway.csv")

In [ ]:
# merge_grp_motorway = df_motorway.groupby(["Month"]).sum()
# # merge_grp_motorway=merge_grp_motorway.to_frame()
# merge_grp_motorway=merge_grp_motorway.reset_index()
# merge_grp_motorway["weekday"]=merge_grp_motorway["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})
# merge_grp_motorway

In [ ]:
# merge_grp_motorway.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\sum_month\motorway\sum_month13_motorway.csv")

In [ ]:
# merge_grp_motorway = df_motorway.groupby(["Month"]).mean()
# # merge_grp_motorway=merge_grp_motorway.to_frame()
# merge_grp_motorway=merge_grp_motorway.reset_index()
# merge_grp_motorway["weekday"]=merge_grp_motorway["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})

In [ ]:
# merge_grp_motorway = df_motorway.groupby(["TNO Trip Hour"]).mean()
# # merge_grp_motorway=merge_grp_motorway.to_frame()
# merge_grp_motorway=merge_grp_motorway.reset_index()
# merge_grp_motorway["weekday"]=merge_grp_motorway["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})

In [ ]:
# merge_grp_motorway.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\mean_hour\motorway\mean_hour13_motorway.csv")

#### Extra Urban

In [ ]:
# merge_grp_extra_urban = df_extra_urban.groupby(["Month"]).sum()
# # merge_grp_extra_urban=merge_grp_extra_urban.to_frame()
# merge_grp_extra_urban=merge_grp_extra_urban.reset_index()
# merge_grp_extra_urban["weekday"]=merge_grp_extra_urban["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})

In [ ]:
# merge_grp_extra_urban.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\sum_month\extra_urban\sum_month13_extra_urban.csv"

In [ ]:
# merge_grp_extra_urban = df_extra_urban.groupby(["Month"]).mean()
# # merge_grp_extra_urban=merge_grp_extra_urban.to_frame()
# merge_grp_extra_urban=merge_grp_extra_urban.reset_index()
# merge_grp_extra_urban["weekday"]=merge_grp_extra_urban["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})

In [ ]:
# merge_grp_extra_urban.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\mean_month\extra_urban\mean_month13_extra_urban.csv"

In [ ]:
# merge_grp_extra_urban = df_extra_urban.groupby(["TNO Trip Hour"]).mean()
# # merge_grp_extra_urban=merge_grp_extra_urban.to_frame()
# merge_grp_extra_urban=merge_grp_extra_urban.reset_index()
# merge_grp_extra_urban["weekday"]=merge_grp_extra_urban["weekday"].replace({0: "Monday", 1: "Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"})

In [ ]:
# merge_grp_extra_urban.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\mean_hour\extra_urban\mean_hour13_extra_urban.csv"

In [ ]:
merge_grp=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\Point Speed\extra_urban\Point_Speed_hour_weekday1_extra_urban.csv")
merge_grp

In [ ]:
ax=sns.lineplot(x=merge_grp["TNO Trip Hour"],y=merge_grp["Point_Speed_extra_urban"],hue=merge_grp["weekday"],linewidth=3.5,palette="plasma")
plt.xlabel("Hour of the day")
plt.ylabel("Mean Point Speed (km/h)")
ax.legend(title="Day",fontsize=9)
ax.legend(bbox_to_anchor=(1.30, 1), loc='upper right', borderaxespad=0.,title="Day",fontsize=10)
plt.title("Road Type - Motorway")
# plt.xlim(-0.5, 23.5)
# plt.ylim(60,80)
plt.yticks([50,55,60,65,70,75,80,85,90])


In [ ]:
tips = sns.load_dataset("tips")
tips

In [ ]:
point_speed_3_roads=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\Point Speed\point_Speed_3_roads.csv")
point_speed_3_roads

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns


sns.set_palette("plasma")
# g=sns.set(rc={'figure.figsize':(25,25)})
g = sns.FacetGrid(point_speed_3_roads, col="Road Type", hue="Day")
g.map(sns.lineplot, "Hour of the day", "Mean Point Speed (km/h)",linewidth=3)
g.add_legend()
plt.xlabel("Hour of the day")
plt.figure(figsize=(30,15))

In [ ]:
cat_weekdayhour = plt.figure(figsize=(20,20))
cat_weekdayhour = sns.set_context("paper")
cat_weekdayhour = sns.set(style="darkgrid", font_scale=1.0)

# weekdayhour.shape
cat_weekdayhour = sns.catplot(x="weekday", y="Point_speed", hue="TNO Trip Hour", kind="swarm", palette="plasma", data=merge_grp)
plt.xlabel("Hour of the day")
plt.ylabel("Point Speed (km/h)")

#### Heatmap Point Speed

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

array = [[77.22471574,77.00576039,77.04276253,76.66354794,74.73674937,70.26279195,64.99583406,64.11780805,67.32261922,67.4417012,67.92093776,68.50080438,68.20613442,68.54404583,68.01214693,67.30968616,68.29744322,71.67173326,74.74030784,76.04037086,76.94580085,77.46864638,77.6337138	,77.54019212],
         [77.58487678,77.12636287,77.35606058,76.62639839,74.97134469,70.71204615,64.65517226,62.37591014,65.13490569,66.24463835,66.81356514,67.35355036,67.28952508,67.79335749,67.14422461,65.73402689,66.10111251,70.44871094,73.99677083,75.7210203,76.98769201,77.28153685,77.79414812,77.66129885],
         [77.71227616,77.32505013,77.11338543,76.52528385,75.07151652,71.51354144,66.39985513,64.19732536,65.98183477,66.70262755,67.12801674,67.64410009,67.65705701,67.78497747,67.14781776,65.49176472,66.40422316,70.69216089,74.23322216,76.22591126,77.0432851,77.22845356,77.53349755,77.63168969],
         [77.41244795,77.32615656,77.12885478,76.77565419,74.99209271,71.11471833,65.25202936,62.85999191,64.97971213,66.59077725,66.93169665,67.66745195,67.58621413,67.37432462,66.3065311,64.91929278,65.46313308,69.24527448,73.50462848,75.06979324,76.5818323,77.20799775,78.04440063,77.64564254],
         [77.85786061,77.63882374,77.10733693,76.4463634,75.26956079,72.66602215,68.72508737,66.24742181,66.79307725,66.56816482,66.32152526,66.42297562,66.32069838,65.64733442,65.28090003,65.61329746,67.49609688,70.64440015,73.2811676,74.913803,76.2048574,77.04932829,76.94008507,77.06635231],
         [77.68160878,76.94236289,74.29844095,74.74768892,75.17134319,73.88660436,72.89003509,71.74867695,71.67956774,70.83777232,71.251072,71.52631129,71.35013599,71.77554966,71.66986874,71.91587171,72.87131457,74.29727998,74.58549105,75.55970443,75.74164292,77.71788856,77.62651907,75.93864547],
         [78.16621971,78.12108976,77.27216559,77.08828218,77.21876289,76.27232365,73.85003252,72.44362085,72.08952198,72.50421241,72.86943397,72.12960513,72.09311982,71.60853231,71.43704883,72.81159132,74.92322085,77.11530333,76.84421714,78.12607556,77.90170889,78.06509106,77.69170247,78.1283126]]

index=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
columns=["0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23"]
# labels = ["Eindhoven","Rotterdam","Amsterdam","Utrecht","Zwolle"]
df_cm = pd.DataFrame(array, index = index,
                  columns = columns)

# matrix = np.triu(df_cm)

plt.figure(figsize = (10,7))
# plt.title('')
sn.heatmap(df_cm, annot=False,fmt=".1f", 
           linewidths=0.5, cmap="coolwarm",cbar_kws={'label': 'Mean Point Speed (km/h)'})
plt.xlabel('Hour of the day',fontweight="bold",fontsize=13)
plt.ylabel('Day of the week',fontweight="bold",fontsize=13)
plt.title(r"Heatmap depicting change in mean point speed (km/h)"+"\n"+r"(over different days and hours) - All road types",fontsize=13,fontweight="bold")

https://github.com/lpegram85/Python_Queries/blob/master/Visualization_Correlation_Matrix.py

In [ ]:
distance_travelled=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\sum_hour\total_sum_hour_road_type.csv")
distance_travelled

In [ ]:
g=sns.lineplot(x=distance_travelled["TNO Trip Hour"],y=distance_travelled["km_sum"],hue=distance_travelled['road_type'],linewidth=3.5,palette="plasma")
plt.xlabel("Hour of the day")
plt.ylabel("Distance Covered (km)")
g.get_legend().remove()
# ax.legend(title="Road Type")
# ax.legend(bbox_to_anchor=(1.05, 1), borderaxespad=0.,title="Road Type",fontsize=10)
# plt.title("Road Type - Urban")
# plt.xlim(-0.5, 23.5)
# plt.ylim(60,80)
# plt.yticks([50,55,60,65,70,75,80,85,90])

In [ ]:
ax=sns.lineplot(x=distance_travelled["TNO Trip Hour"],y=distance_travelled["km_sum"],hue=distance_travelled['road_type'],linewidth=3.5,palette="plasma")
plt.xlabel("Hour of the day")
plt.ylabel("Distance Covered (km)")
ax.legend(title="Road Type")

In [ ]:
ax=sns.barplot(x=distance_travelled["TNO Trip Hour"],y=distance_travelled["km_sum"],hue=distance_travelled['road_type'],linewidth=3.5,palette="plasma")
plt.xlabel("Hour of the day")
plt.ylabel("Distance Covered (kilometers)")

#### AOS events plot

In [ ]:
import datashader as ds, pandas as pd, colorcet as cc

fields = ['Latitude', 'Longitude','Event/action_type']
df = pd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_SUMMARY.csv',delimiter=';', skipinitialspace=True, usecols=fields, encoding= 'unicode_escape',dtype={'Latitude': float,'Longitude':float},skiprows=range(1,2))

In [ ]:
import numpy as np
import holoviews as hv
from holoviews import opts
from holoviews.element.tiles import StamenTerrain
hv.extension('bokeh')

In [ ]:
from bokeh.models import BoxZoomTool
from bokeh.plotting import figure, output_notebook, show

output_notebook()

# Define general parameters for the Bokeh plot
x_range, y_range = ((-1477374.882696,3639625.538827), (4197310.097196,9646964.465816))

plot_width  = int(990)
plot_height = int(plot_width//1.2)

options = dict(line_color=None, fill_color='blue', size=5)

In [ ]:
opts.defaults(
    opts.Points(width=plot_width, height=plot_height, size=5, color='blue'),
    opts.Overlay(width=plot_width, height=plot_height, xaxis=None, yaxis=None),
    opts.RGB(width=plot_width, height=plot_height))

In [ ]:
samples = df.sample(frac=1e-4)
tiles = StamenTerrain().redim.range(x=x_range, y=y_range)
points = hv.Points(samples, ['Longitude', 'Latitude'])
(tiles * points)

In [ ]:
import datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import Greys9
Greys9_r = list(reversed(Greys9))[:-2]

In [ ]:
%%time
cvs = ds.Canvas(plot_width=plot_width, plot_height=plot_height, x_range=x_range, y_range=y_range)
agg = cvs.points(df, 'Longitude', 'Latitude')
img = tf.shade(agg, cmap=["white", 'darkblue'], how='linear')

In [ ]:
img

In [ ]:
tf.shade(agg, cmap=Greys9_r, how='log')

In [ ]:
tf.shade(agg, cmap=Greys9_r, how='eq_hist')

In [ ]:
import holoviews.operation.datashader as hd
from datashader.colors import Hot
shaded = hd.datashade(hv.Points(df, ['Longitude', 'Latitude']), cmap=Hot, aggregator=ds.count('Event/action_type'))
hd.dynspread(shaded, threshold=0.5, max_px=4).opts(bgcolor='black', xaxis=None, yaxis=None, width=900, height=500)

In [ ]:
def transform(overlay):
    picks = overlay.get(0).redim(Longitude='x', Latitude='y')
    events=
    # drops = overlay.get(1).redim(dropoff_x='x', dropoff_y='y')
    # pick_agg = picks.data.Count.data
    # drop_agg = drops.data.Count.data
    # more_picks = picks.clone(picks.data.where(pick_agg>drop_agg))
    # more_drops = drops.clone(drops.data.where(drop_agg>pick_agg))
    return (hd.shade(picks, cmap=['lightcyan', "blue"]) *
            hd.shade(more_picks, cmap=['mistyrose', "red"]))

picks = hv.Points(df, ['pickup_x',  'pickup_y'])
drops = hv.Points(df, ['dropoff_x', 'dropoff_y'])
((hd.rasterize(picks) * hd.rasterize(drops))).apply(transform).opts(
    bgcolor='white', xaxis=None, yaxis=None, width=900, height=500)

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_SUMMARY.csv',delimiter=';', encoding='unicode_escape',usecols=["Latitude","Longitude"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
# df = df[df['Road_type']==0]
with ProgressBar():
    df = df.compute()
    
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\AOS_warnings_all.csv")

In [ ]:
len(df)

In [ ]:
df["Latitude"] = df["Latitude"].astype(float)
df["Longitude"] = df["Longitude"].astype(float)

In [ ]:
df=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\headway_warnings.csv")

In [ ]:
len(df)

In [ ]:
from bokeh.models import BoxZoomTool
from bokeh.plotting import figure, output_notebook, show

output_notebook()

# Define general parameters for the Bokeh plot
NYC = x_range, y_range = ((-1477374.882696,3639625.538827), (4197310.097196,9646964.465816))

plot_width  = int(990)
plot_height = int(plot_width//1.2)

options = dict(line_color=None, fill_color='blue', size=5)

In [ ]:
# Reusable function to create simple Bokeh plots

def base_plot(tools='pan,wheel_zoom,reset', plot_width=plot_width, plot_height=plot_height, **plot_kwargs):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_kwargs)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    p.add_tools(BoxZoomTool(match_aspect=True))
    
    return p

In [ ]:
from datashader.utils import lnglat_to_meters
df.loc[:, 'Longitude'], df.loc[:, 'Latitude'] = lnglat_to_meters(df.Longitude,df.Latitude)

In [ ]:
from bokeh.tile_providers import get_provider, Vendors 

tile_provider = get_provider(Vendors.CARTODBPOSITRON)

samples = df.sample(n=100000)
p = base_plot()
p.add_tile(tile_provider)

p.circle(x=samples['Longitude'], y=samples['Latitude'], **options)
show(p)

In [ ]:
from bokeh.tile_providers import get_provider, Vendors 
import holoviews.operation.datashader as hd
from datashader.colors import Hot

tile_provider = get_provider(Vendors.CARTODBPOSITRON)
options = dict(line_color=None, fill_color='blue', size=1, alpha=0.2)
samples = df.sample(n=1000000)
p = base_plot()
p.add_tile(tile_provider)
p.circle(x=samples['Longitude'], y=samples['Latitude'], **options)
show(p)

In [ ]:
from bokeh.io import export_png

export_png(p, filename="plot.png")

In [ ]:
from bokeh.io import export_png

export_png(p, filename="brake_on.png")

In [ ]:
import seaborn as sns
#load the dataset
df = sns.load_dataset('iris')
df
# #calculate correlation
# corr_matrix = df.corr('pearson') #kind of correlation->  ‘pearson’, ‘kendall’, ‘spearman’
# #plot correlation
# corr_matrix.style.background_gradient(cmap='coolwarm')
# # 'RdBu_r', 'BrBG_r', & PuOr_r are other good diverging colormaps

In [ ]:
df_hour=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\Overview\Point Speed\point_Speed_3_roads.csv")
df_hour

In [ ]:
corr_matrix = df_hour.corr('pearson') #kind of correlation->  ‘pearson’, ‘kendall’, ‘spearman’
corr_matrix

In [ ]:
corr_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
corr_matrix = df.corr('pearson') #kind of correlation->  ‘pearson’, ‘kendall’, ‘spearman’
corr_matrix

In [ ]:
corr_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
df_td = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', encoding='unicode_escape', usecols=['TNO_Time-stamp','Point_speed'])
df_td

In [ ]:
fields = ['Point_speed']
df = pd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', skipinitialspace=True, usecols=fields, encoding= 'unicode_escape',dtype={'Point_speed': float},skiprows=range(1,2),nrows=10000000)

In [ ]:
fields = ['Speed_restriction']
df = pd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\TRIP_DETAIL.csv',delimiter=';', skipinitialspace=True, usecols=fields, encoding= 'unicode_escape',dtype={'Speed_restriction': float},skiprows=range(1,2),nrows=55000000)

In [ ]:
fields = ['Point_speed','Average_speed_fpp','Road_type_Rename']
df = pd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv', skipinitialspace=True, usecols=fields, encoding= 'unicode_escape',dtype={'Point_speed': float,'Average_speed_fpp':float,'Road_type_Rename':str},skiprows=range(1,2))

In [ ]:
df = df.drop(df[df['Average_speed_fpp']==0].index)
df = df.drop(df[df['Point_speed']==0].index)
df = df.drop(df[df['Point_speed']<20].index)
df = df.drop(df[df['Point_speed']>140].index)


df = df.drop(df[df['Road_type_Rename']=="Unavailable"].index)
df = df.drop(df[df['Road_type_Rename']=="-2"].index)
# df = df.drop(df[df['Road_type_Rename']>120].index)

In [ ]:
df['Road_type_Rename'].unique()

In [ ]:
sns.kdeplot(data=df, x="Point_speed",palette="crest")
plt.xlabel("Point Speed (km/h)")
plt.title("Distribution of Point Speed")

In [ ]:
# df=cwReturnVehiclePositions_4750[cwReturnVehiclePositions_4750['ReceivedTime_Sep']=='2008-08-08']
import joypy
from matplotlib import cm

joypy.joyplot(df, column=['Point_speed'], by="Road_type_Rename", ylim='own',linecolor="blue", colormap=cm.autumn_r,title="Distribution of Point Speed on different road types")

In [ ]:
joypy.joyplot(df,column=['Point_speed'], by="Road_type_Rename", ylim='own',fill=False, colormap=cm.coolwarm,title="Distribution of Point Speed on different road types",linewidth=2.5)


In [ ]:
joypy.joyplot(df,column=['Point_speed'], by="Road_type_Rename", ylim='own',fill=False, colormap=cm.CMRmap,title="Distribution of Point Speed on different road types",linewidth=2.5)

In [ ]:
joypy.joyplot(df,column=['Point_speed'], by="Road_type_Rename", ylim='own',fill=False, colormap=cm.PuRd,title="Distribution of Point Speed on different road types",linewidth=2.5)

In [ ]:
# df=cwReturnVehiclePositions_4750[cwReturnVehiclePositions_4750['ReceivedTime_Sep']=='2008-08-08']
import joypy
from matplotlib import cm

joypy.joyplot(df, column=['Point_speed'], by="Road_type_Rename", ylim='own',linecolor="blue", colormap=cm.PuRd,title="Distribution of Point Speed on different road types")

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df20M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_20M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df40M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_40M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df60M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_60M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df80M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_80M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df100M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_100M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df120M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_120M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df140M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_140M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df160M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_160M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df180M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_180M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df200M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_200M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df220M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_220M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df240M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_240M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df260M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_260M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\df280M.csv', encoding='unicode_escape',assume_missing=True,usecols=["Point_speed","Numberplate","TNO_Time-stamp","TNO_Time-stamp_hour"])

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==1]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1\hour1_TD_280M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

stop

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_20M.csv', encoding='unicode_escape',assume_missing=True)

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==0]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour\Hour0\hour0_AOS_20M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

this

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_40M.csv', encoding='unicode_escape',assume_missing=True)

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==0]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour\Hour0\hour0_AOS_40M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_60M.csv', encoding='unicode_escape',assume_missing=True)

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==0]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour\Hour0\hour0_AOS_60M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_80M.csv', encoding='unicode_escape',assume_missing=True)

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==0]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour\Hour0\hour0_AOS_80M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_100M.csv', encoding='unicode_escape',assume_missing=True)

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==0]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour\Hour0\hour0_AOS_100M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_120M.csv', encoding='unicode_escape',assume_missing=True)

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==0]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour\Hour0\hour0_AOS_120M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

In [ ]:

df = dd.read_csv(r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\aos_summary_140M.csv', encoding='unicode_escape',assume_missing=True)

# headway_warning=[11,12,13]
# df = df[df['Event/action_type'].isin(headway_warning)]
df = df[df['TNO_Time-stamp_hour']==0]
with ProgressBar():
    df = df.compute()
    
df.to_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour\Hour0\hour0_AOS_140M.csv")
# df['platedate'] = df['Numberplate'] + df['TNO_Time-stamp'].str[:10]

#### Temporal Stability

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour0'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour0 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour0 = df_TD_hour0.drop(df_TD_hour0[df_TD_hour0['Point_speed']==0].index)
df_TD_hour0.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour0.rename(columns={'Point_speed':'0'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour0['Count'] = np.arange(len(df_TD_hour0))

df_TD_hour0


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour1 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour1 = df_TD_hour1.drop(df_TD_hour1[df_TD_hour1['Point_speed']==0].index)
df_TD_hour1.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour1.rename(columns={'Point_speed':'1'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour1['Count'] = np.arange(len(df_TD_hour1))

df_TD_hour1


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour2'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour2 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour2 = df_TD_hour2.drop(df_TD_hour2[df_TD_hour2['Point_speed']==0].index)
df_TD_hour2.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour2.rename(columns={'Point_speed':'2'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour2['Count'] = np.arange(len(df_TD_hour2))
df_TD_hour2


In [ ]:
sns.regplot(r)

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour3'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour3 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour3 = df_TD_hour3.drop(df_TD_hour3[df_TD_hour3['Point_speed']==0].index)
df_TD_hour3.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour3.rename(columns={'Point_speed':'3'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour3['Count'] = np.arange(len(df_TD_hour3))

df_TD_hour3


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour4'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour4 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour4 = df_TD_hour4.drop(df_TD_hour4[df_TD_hour4['Point_speed']==0].index)
df_TD_hour4.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour4.rename(columns={'Point_speed':'4'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour4['Count'] = np.arange(len(df_TD_hour4))

df_TD_hour4


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour5'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour5 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour5 = df_TD_hour5.drop(df_TD_hour5[df_TD_hour5['Point_speed']==0].index)
df_TD_hour5.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour5.rename(columns={'Point_speed':'5'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour5['Count'] = np.arange(len(df_TD_hour5))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour6'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour6 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour6 = df_TD_hour6.drop(df_TD_hour6[df_TD_hour6['Point_speed']==0].index)
df_TD_hour6.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour6.rename(columns={'Point_speed':'6'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour6['Count'] = np.arange(len(df_TD_hour6))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour7'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour7 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour7 = df_TD_hour7.drop(df_TD_hour7[df_TD_hour7['Point_speed']==0].index)
df_TD_hour7.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour7.rename(columns={'Point_speed':'7'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour7['Count'] = np.arange(len(df_TD_hour7))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour8'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour8 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour8 = df_TD_hour8.drop(df_TD_hour8[df_TD_hour8['Point_speed']==0].index)
df_TD_hour8
# df_TD_hour8.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
# df_TD_hour8.rename(columns={'Point_speed':'8'},
        #   inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
# df_TD_hour8['Count'] = np.arange(len(df_TD_hour8))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour9'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour9 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour9 = df_TD_hour9.drop(df_TD_hour9[df_TD_hour9['Point_speed']==0].index)
# df_TD_hour9.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
# df_TD_hour9.rename(columns={'Point_speed':'9'},
#           inplace=True, errors='raise')
# # df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
# df_TD_hour9['Count'] = np.arange(len(df_TD_hour9))

df_TD_hour9


In [ ]:
common_drivers=(np.intersect1d(df_TD_hour8['Numberplate'], df_TD_hour9['Numberplate']))

In [ ]:
df_TD_hour8 = df_TD_hour8[df_TD_hour8['Numberplate'].isin(common_drivers)]
df_TD_hour8 = df_TD_hour8.groupby(['Numberplate']).agg(Point_speed_hourly_average_8 = ('Point_speed', 'mean'))
df_TD_hour8.reset_index(inplace=True)
# df_TRIP_DETAIL_Motorway_10['Hour'] = '14'
# df_TD_hour8

df_TD_hour9 = df_TD_hour9[df_TD_hour9['Numberplate'].isin(common_drivers)]
df_TD_hour9 = df_TD_hour9.groupby(['Numberplate']).agg(Point_speed_hourly_average_9 = ('Point_speed', 'mean'))
df_TD_hour9.reset_index(inplace=True)

# df_TD_hour9
# df_TRIP_DETAIL_Motorway_9['Hour'] = '15'

In [ ]:
df_8thand9thHour = pd.merge(df_TD_hour8, df_TD_hour9, on='Numberplate', how='inner') #here id is common column

df_8thand9thHour

In [ ]:
#  import pandas as pd 
#  import seaborn as sns
#  import matplotlib.pyplot as plt 
# import stats
from scipy import stats

p = sns.lmplot(x="Point_speed_hourly_average_8", y="Point_speed_hourly_average_9",
        data=df_8thand9thHour,scatter_kws={"color": "cornflowerblue"},
        line_kws={'label':"Linear Reg","color":"cornflowerblue"}, legend=True)

ax = p.axes[0, 0]
ax.legend()
leg = ax.get_legend()
L_labels = leg.get_texts()
# assuming you computed r_squared which is the coefficient of determination somewhere else
slope, intercept, r_value, p_value, std_err = stats.linregress(df_8thand9thHour["Point_speed_hourly_average_8"],df_8thand9thHour["Point_speed_hourly_average_9"])
label_line_1 = r'$y={0:.1f}x+{1:.1f}'.format(slope,intercept)
label_line_2 = r'$r:{0:.2f}$'.format(r_value) # as an exampple or whatever you want[!
# L_labels[0].set_text(label_line_1)
L_labels[0].set_text(label_line_2)
# plt.title("Normalized Braking Events - Amsterdam & the NL (Urban Roads)")

In [ ]:
df_TRIP_DETAIL_Motorway_9=df_TRIP_DETAIL_Motorway[df_TRIP_DETAIL_Motorway['TNO_Time-stamp_hour']==15]
df_TRIP_DETAIL_Motorway_10=df_TRIP_DETAIL_Motorway[df_TRIP_DETAIL_Motorway['TNO_Time-stamp_hour']==14]

common_drivers=(np.intersect1d(df_TRIP_DETAIL_Motorway_9['Numberplate'], df_TRIP_DETAIL_Motorway_10['Numberplate']))

df_TRIP_DETAIL_Motorway_10 = df_TRIP_DETAIL_Motorway_10[df_TRIP_DETAIL_Motorway_10['Numberplate'].isin(common_drivers)]
df_TRIP_DETAIL_Motorway_10['Hour'] = '14'


df_TRIP_DETAIL_Motorway_9 = df_TRIP_DETAIL_Motorway_9[df_TRIP_DETAIL_Motorway_9['Numberplate'].isin(common_drivers)]
df_TRIP_DETAIL_Motorway_9['Hour'] = '15'


df_TRIP_DETAIL_Motorway_9 = df_TRIP_DETAIL_Motorway_9.groupby(['Numberplate']).agg(Point_speed_hourly_average = ('Point_speed', 'mean'))
df_TRIP_DETAIL_Motorway_9['Hour']='15'

df_TRIP_DETAIL_Motorway_10 = df_TRIP_DETAIL_Motorway_10.groupby(['Numberplate']).agg(Point_speed_hourly_average = ('Point_speed', 'mean'))
df_TRIP_DETAIL_Motorway_10['Hour']='14'

frames=[df_TRIP_DETAIL_Motorway_9,df_TRIP_DETAIL_Motorway_10]
df_9thand10thHour=pd.concat(frames)


df_9thand10thHour = df_9thand10thHour.drop(df_9thand10thHour[df_9thand10thHour['Point_speed_hourly_average']==0].index)

x_axis=df_9thand10thHour[df_9thand10thHour['Hour']=='15']['Point_speed_hourly_average']
y_axis=df_9thand10thHour[df_9thand10thHour['Hour']=='14']['Point_speed_hourly_average']

sns.regplot(x=y_axis, y=x_axis, data=df_9thand10thHour)
plt.title('Point Speed of drivers on Motorways \n (First 2 million Datapoints) \n Correlation Coefficient : 0.323')
plt.xlabel('Mean Point Speed (km/h) at 2 pm')
plt.ylabel('Mean Point Speed (km/h) at 3 pm')

# df_9thand10thHour

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour9'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour9 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour9 = df_TD_hour9.drop(df_TD_hour9[df_TD_hour9['Point_speed']==0].index)
df_TD_hour9.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour9.rename(columns={'Point_speed':'9'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour9['Count'] = np.arange(len(df_TD_hour9))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour10'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour10 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour10 = df_TD_hour10.drop(df_TD_hour10[df_TD_hour10['Point_speed']==0].index)
df_TD_hour10.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour10.rename(columns={'Point_speed':'10'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour10['Count'] = np.arange(len(df_TD_hour10))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour11'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour11 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour11 = df_TD_hour11.drop(df_TD_hour11[df_TD_hour11['Point_speed']==0].index)
df_TD_hour11.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour11.rename(columns={'Point_speed':'11'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour11['Count'] = np.arange(len(df_TD_hour11))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour12'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour12 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour12 = df_TD_hour12.drop(df_TD_hour12[df_TD_hour12['Point_speed']==0].index)
df_TD_hour12.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour12.rename(columns={'Point_speed':'12'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour12['Count'] = np.arange(len(df_TD_hour12))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour13'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour13 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour13 = df_TD_hour13.drop(df_TD_hour13[df_TD_hour13['Point_speed']==0].index)
df_TD_hour13.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour13.rename(columns={'Point_speed':'13'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour13['Count'] = np.arange(len(df_TD_hour13))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour14'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour14 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour14 = df_TD_hour14.drop(df_TD_hour14[df_TD_hour14['Point_speed']==0].index)
df_TD_hour14.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour14.rename(columns={'Point_speed':'14'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour14['Count'] = np.arange(len(df_TD_hour14))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour15'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour15 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour15 = df_TD_hour15.drop(df_TD_hour15[df_TD_hour15['Point_speed']==0].index)
df_TD_hour15.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour15.rename(columns={'Point_speed':'15'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour15['Count'] = np.arange(len(df_TD_hour15))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour16'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour16 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour16 = df_TD_hour16.drop(df_TD_hour16[df_TD_hour16['Point_speed']==0].index)
df_TD_hour16.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour16.rename(columns={'Point_speed':'16'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour16['Count'] = np.arange(len(df_TD_hour16))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour17'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour17 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour17 = df_TD_hour17.drop(df_TD_hour17[df_TD_hour17['Point_speed']==0].index)
df_TD_hour17.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour17.rename(columns={'Point_speed':'17'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour17['Count'] = np.arange(len(df_TD_hour17))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour18'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour18 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour18 = df_TD_hour18.drop(df_TD_hour18[df_TD_hour18['Point_speed']==0].index)
df_TD_hour18.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour18.rename(columns={'Point_speed':'18'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour18['Count'] = np.arange(len(df_TD_hour18))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour19'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour19 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour19 = df_TD_hour19.drop(df_TD_hour19[df_TD_hour19['Point_speed']==0].index)
df_TD_hour19.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour19.rename(columns={'Point_speed':'19'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour19['Count'] = np.arange(len(df_TD_hour19))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour20'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour20 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour20 = df_TD_hour20.drop(df_TD_hour20[df_TD_hour20['Point_speed']==0].index)
df_TD_hour20.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour20.rename(columns={'Point_speed':'20'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour20['Count'] = np.arange(len(df_TD_hour20))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour21'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour21 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour21 = df_TD_hour21.drop(df_TD_hour21[df_TD_hour21['Point_speed']==0].index)
df_TD_hour21.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour21.rename(columns={'Point_speed':'21'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour21['Count'] = np.arange(len(df_TD_hour21))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour22'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour22 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour22 = df_TD_hour22.drop(df_TD_hour22[df_TD_hour22['Point_speed']==0].index)
df_TD_hour22.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour22.rename(columns={'Point_speed':'22'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour22['Count'] = np.arange(len(df_TD_hour22))

# df_TD_hour5


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour23'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour23 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour23 = df_TD_hour23.drop(df_TD_hour23[df_TD_hour23['Point_speed']==0].index)
df_TD_hour23.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
df_TD_hour23.rename(columns={'Point_speed':'23'},
          inplace=True, errors='raise')
# df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
df_TD_hour23['Count'] = np.arange(len(df_TD_hour23))

# df_TD_hour5


In [ ]:
import pandas as pd
from functools import reduce

# compile the list of dataframes you want to merge
data_frames = [df_TD_hour0, df_TD_hour1, df_TD_hour2,df_TD_hour3,df_TD_hour4,df_TD_hour5,df_TD_hour6,df_TD_hour7,df_TD_hour8,df_TD_hour9,df_TD_hour10,df_TD_hour11,df_TD_hour12,df_TD_hour13,df_TD_hour14,
               df_TD_hour15,df_TD_hour16,df_TD_hour17,df_TD_hour18,df_TD_hour19,df_TD_hour20,df_TD_hour21,df_TD_hour22,df_TD_hour23]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Count'],
                                            how='inner'), data_frames)

In [ ]:
df_merged

In [ ]:
import seaborn as sns
# cmap =sns.diverging_palette(145, 300, s=60, as_cmap=True)

# sns.color_palette("viridis", as_cmap=True)

sns.set(rc = {'figure.figsize':(15,8)})
sns.heatmap(df_merged.corr(), annot=False,fmt=".1f", 
           linewidths=0.5,cbar_kws={'label': 'Correlation Coefficient'},cmap="Spectral_r")
plt.xlabel('Hour of the day',fontweight="bold",fontsize=13)
plt.ylabel('Hour of the day',fontweight="bold",fontsize=13)
plt.title(r"Heatmap depicting correlation of Point Speed"+"\n"+r"(hours) - All road types",fontsize=13,fontweight="bold")

In [ ]:
# df_outer = pd.merge(df_TD_hour2, df_TD_hour3, on='Count', how='inner') #here Numberplate is common column
# df_merged.drop(['Count'], axis = 1, inplace = True) 
sns.heatmap(df_merged.corr(),vmin=0,vmax=0.3)


In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour2'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour4 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour4 = df_TD_hour4.drop(df_TD_hour4[df_TD_hour4['Point_speed']==0].index)
df_TD_hour4.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp'], axis = 1, inplace = True) 
df_TD_hour4.rename(columns={'TNO_Time-stamp_hour': 'Hour',
                            'Point_speed':'Point Speed'},
          inplace=True, errors='raise')
df_TD_hour4["Hour"] = df_TD_hour4["Hour"].astype(int)
df_TD_hour4


In [ ]:
flights = sns.load_dataset("flights")
flights
# flights = flights.pivot("month", "year", "passengers")
ax = sns.heatmap(flights)

In [ ]:
flights = flights.pivot("month", "year", "passengers")
flights

In [ ]:
# df_TD_hour8 = df_TD_hour8[df_TD_hour8['Numberplate'].isin(common_drivers)]
# df_TD_hour8 = df_TD_hour8.groupby(['Numberplate']).agg(Point_speed_hourly_average_8 = ('Point_speed', 'mean'))
# df_TD_hour8.reset_index(inplace=True)
# df_TRIP_DETAIL_Motorway_10['Hour'] = '14'
# df_TD_hour8

# df_TD_hour9 = df_TD_hour9[df_TD_hour9['Numberplate'].isin(common_drivers)]
df_TD_hour9 = df_TD_hour9.groupby(['Numberplate']).agg(Point_speed_hourly_average_9 = ('Point_speed', 'mean'))
df_TD_hour9.reset_index(inplace=True)

# df_TD_hour9
# df_TRIP_DETAIL_Motorway_9['Hour'] = '15'

Numberplate Temporal Stability

In [ ]:
import glob
import os

import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour0'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour0 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour0 = df_TD_hour0.drop(df_TD_hour0[df_TD_hour0['Point_speed']==0].index)
df_TD_hour0 = df_TD_hour0.groupby(['Numberplate']).agg(Point_speed_hourly_average_0 = ('Point_speed', 'mean'))
df_TD_hour0.reset_index(inplace=True)


In [ ]:
df_TD_hour0

In [ ]:
path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour1'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour1 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour1 = df_TD_hour1.drop(df_TD_hour1[df_TD_hour1['Point_speed']==0].index)
df_TD_hour1 = df_TD_hour1.groupby(['Numberplate']).agg(Point_speed_hourly_average_1 = ('Point_speed', 'mean'))
df_TD_hour1.reset_index(inplace=True)

In [ ]:
path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour2'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour2 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour2 = df_TD_hour2.drop(df_TD_hour2[df_TD_hour2['Point_speed']==0].index)
df_TD_hour2 = df_TD_hour2.groupby(['Numberplate']).agg(Point_speed_hourly_average_2 = ('Point_speed', 'mean'))
df_TD_hour2.reset_index(inplace=True)

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour3'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour3 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour3 = df_TD_hour3.drop(df_TD_hour3[df_TD_hour3['Point_speed']==0].index)
df_TD_hour3 = df_TD_hour3.groupby(['Numberplate']).agg(Point_speed_hourly_average_3 = ('Point_speed', 'mean'))
df_TD_hour3.reset_index(inplace=True)


In [ ]:

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour4'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour4 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour4 = df_TD_hour4.drop(df_TD_hour4[df_TD_hour4['Point_speed']==0].index)
df_TD_hour4 = df_TD_hour4.groupby(['Numberplate']).agg(Point_speed_hourly_average_4 = ('Point_speed', 'mean'))
df_TD_hour4.reset_index(inplace=True)


In [ ]:

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour5'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour5 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour5 = df_TD_hour5.drop(df_TD_hour5[df_TD_hour5['Point_speed']==0].index)
df_TD_hour5 = df_TD_hour5.groupby(['Numberplate']).agg(Point_speed_hourly_average_5 = ('Point_speed', 'mean'))
df_TD_hour5.reset_index(inplace=True)

In [ ]:

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour6'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour6 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour6 = df_TD_hour6.drop(df_TD_hour6[df_TD_hour6['Point_speed']==0].index)
df_TD_hour6 = df_TD_hour6.groupby(['Numberplate']).agg(Point_speed_hourly_average_6 = ('Point_speed', 'mean'))
df_TD_hour6.reset_index(inplace=True)

In [ ]:
path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour7'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour7 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour7 = df_TD_hour7.drop(df_TD_hour7[df_TD_hour7['Point_speed']==0].index)
df_TD_hour7 = df_TD_hour7.groupby(['Numberplate']).agg(Point_speed_hourly_average_7 = ('Point_speed', 'mean'))
df_TD_hour7.reset_index(inplace=True)

In [ ]:
path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour8'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour8 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour8 = df_TD_hour8.drop(df_TD_hour8[df_TD_hour8['Point_speed']==0].index)
df_TD_hour8 = df_TD_hour8.groupby(['Numberplate']).agg(Point_speed_hourly_average_8 = ('Point_speed', 'mean'))
df_TD_hour8.reset_index(inplace=True)

In [ ]:

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour9'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour9 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour9 = df_TD_hour9.drop(df_TD_hour9[df_TD_hour9['Point_speed']==0].index)
df_TD_hour9 = df_TD_hour9.groupby(['Numberplate']).agg(Point_speed_hourly_average_9 = ('Point_speed', 'mean'))
df_TD_hour9.reset_index(inplace=True)

In [ ]:
path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour10'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour10 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour10 = df_TD_hour10.drop(df_TD_hour10[df_TD_hour10['Point_speed']==0].index)
df_TD_hour10 = df_TD_hour10.groupby(['Numberplate']).agg(Point_speed_hourly_average_10 = ('Point_speed', 'mean'))
df_TD_hour10.reset_index(inplace=True)


In [ ]:

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour11'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour11 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour11 = df_TD_hour11.drop(df_TD_hour11[df_TD_hour11['Point_speed']==0].index)
df_TD_hour11 = df_TD_hour11.groupby(['Numberplate']).agg(Point_speed_hourly_average_11 = ('Point_speed', 'mean'))
df_TD_hour11.reset_index(inplace=True)

In [ ]:


path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour12'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour12 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour12 = df_TD_hour12.drop(df_TD_hour12[df_TD_hour12['Point_speed']==0].index)
df_TD_hour12 = df_TD_hour12.groupby(['Numberplate']).agg(Point_speed_hourly_average_12 = ('Point_speed', 'mean'))
df_TD_hour12.reset_index(inplace=True)

In [ ]:

path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour13'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour13 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour13 = df_TD_hour13.drop(df_TD_hour13[df_TD_hour13['Point_speed']==0].index)
df_TD_hour13 = df_TD_hour13.groupby(['Numberplate']).agg(Point_speed_hourly_average_13 = ('Point_speed', 'mean'))
df_TD_hour13.reset_index(inplace=True)

In [ ]:
path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour14'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour14 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour14 = df_TD_hour14.drop(df_TD_hour14[df_TD_hour14['Point_speed']==0].index)
df_TD_hour14 = df_TD_hour14.groupby(['Numberplate']).agg(Point_speed_hourly_average_14 = ('Point_speed', 'mean'))
df_TD_hour14.reset_index(inplace=True)

In [ ]:


path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour15'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour15 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour15 = df_TD_hour15.drop(df_TD_hour15[df_TD_hour15['Point_speed']==0].index)
df_TD_hour15 = df_TD_hour15.groupby(['Numberplate']).agg(Point_speed_hourly_average_15 = ('Point_speed', 'mean'))
df_TD_hour15.reset_index(inplace=True)

In [ ]:
path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour16'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour16 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour16 = df_TD_hour16.drop(df_TD_hour16[df_TD_hour16['Point_speed']==0].index)
df_TD_hour16 = df_TD_hour16.groupby(['Numberplate']).agg(Point_speed_hourly_average_16 = ('Point_speed', 'mean'))
df_TD_hour16.reset_index(inplace=True)


In [ ]:


path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour17'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour17 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour17 = df_TD_hour17.drop(df_TD_hour17[df_TD_hour17['Point_speed']==0].index)
df_TD_hour17 = df_TD_hour17.groupby(['Numberplate']).agg(Point_speed_hourly_average_17 = ('Point_speed', 'mean'))
df_TD_hour17.reset_index(inplace=True)

In [ ]:


path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour18'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour18 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour18 = df_TD_hour18.drop(df_TD_hour18[df_TD_hour18['Point_speed']==0].index)
df_TD_hour18 = df_TD_hour18.groupby(['Numberplate']).agg(Point_speed_hourly_average_18 = ('Point_speed', 'mean'))
df_TD_hour18.reset_index(inplace=True)

In [ ]:


path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour19'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour19 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour19 = df_TD_hour19.drop(df_TD_hour19[df_TD_hour19['Point_speed']==0].index)
df_TD_hour19 = df_TD_hour19.groupby(['Numberplate']).agg(Point_speed_hourly_average_19 = ('Point_speed', 'mean'))
df_TD_hour19.reset_index(inplace=True)

In [ ]:


path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour20'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour20 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour20 = df_TD_hour20.drop(df_TD_hour20[df_TD_hour20['Point_speed']==0].index)
df_TD_hour20 = df_TD_hour20.groupby(['Numberplate']).agg(Point_speed_hourly_average_20 = ('Point_speed', 'mean'))
df_TD_hour20.reset_index(inplace=True)

In [ ]:



path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour21'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour21 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour21 = df_TD_hour21.drop(df_TD_hour21[df_TD_hour21['Point_speed']==0].index)
df_TD_hour21 = df_TD_hour21.groupby(['Numberplate']).agg(Point_speed_hourly_average_21 = ('Point_speed', 'mean'))
df_TD_hour21.reset_index(inplace=True)

In [ ]:


path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour22'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour22 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour22 = df_TD_hour22.drop(df_TD_hour22[df_TD_hour22['Point_speed']==0].index)
df_TD_hour22 = df_TD_hour22.groupby(['Numberplate']).agg(Point_speed_hourly_average_22 = ('Point_speed', 'mean'))
df_TD_hour22.reset_index(inplace=True)

In [ ]:


path=r'D:\AOS FOT\Octo\CSV Export\TRIP_DETAIL\split_data\Hour\Hour23'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_TD_hour23 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
df_TD_hour23 = df_TD_hour23.drop(df_TD_hour23[df_TD_hour23['Point_speed']==0].index)
df_TD_hour23 = df_TD_hour23.groupby(['Numberplate']).agg(Point_speed_hourly_average_23 = ('Point_speed', 'mean'))
df_TD_hour23.reset_index(inplace=True)

In [ ]:
import pandas as pd
from functools import reduce

# compile the list of dataframes you want to merge
data_frames = [df_TD_hour0, df_TD_hour1, df_TD_hour2,df_TD_hour3,df_TD_hour4,df_TD_hour5,df_TD_hour6,df_TD_hour7,df_TD_hour8,df_TD_hour9,df_TD_hour10,df_TD_hour11,df_TD_hour12,df_TD_hour13,df_TD_hour14,
               df_TD_hour15,df_TD_hour16,df_TD_hour17,df_TD_hour18,df_TD_hour19,df_TD_hour20,df_TD_hour21,df_TD_hour22,df_TD_hour23]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Numberplate'],
                                            how='inner'), data_frames)

df_merged.rename(columns={'Point_speed_hourly_average_0': '0',
                            'Point_speed_hourly_average_1': '1',
                            'Point_speed_hourly_average_2': '2',
                            'Point_speed_hourly_average_3': '3',
                            'Point_speed_hourly_average_4': '4',
                            'Point_speed_hourly_average_5': '5',
                            'Point_speed_hourly_average_6': '6',
                            'Point_speed_hourly_average_7': '7',
                            'Point_speed_hourly_average_8': '8',
                            'Point_speed_hourly_average_9': '9',
                            'Point_speed_hourly_average_10': '10',
                            'Point_speed_hourly_average_11': '11',
                            'Point_speed_hourly_average_12': '12',
                            'Point_speed_hourly_average_13': '13',
                            'Point_speed_hourly_average_14': '14',
                            'Point_speed_hourly_average_15': '15',
                            'Point_speed_hourly_average_16': '16',
                            'Point_speed_hourly_average_17': '17',
                            'Point_speed_hourly_average_18': '18',
                            'Point_speed_hourly_average_19': '19',
                            'Point_speed_hourly_average_20': '20',
                            'Point_speed_hourly_average_21': '21',
                            'Point_speed_hourly_average_22': '22',
                            'Point_speed_hourly_average_23': '23'},inplace=True, errors='raise')

In [ ]:
df_merged.drop(['Numberplate'], axis = 1, inplace = True) 


In [ ]:
df_merged.corr()

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

# df_cm = pd.DataFrame(array)

# matrix = np.triu(df_cm)

plt.figure(figsize = (10,7))
# plt.title('')
sn.heatmap(df_merged.corr(), annot=False,fmt=".1f", 
           linewidths=0.5, cmap="coolwarm",cbar_kws={'label': 'Mean Point Speed (km/h)'})
plt.xlabel('Hour of the day',fontweight="bold",fontsize=13)
plt.ylabel('Hour of the day',fontweight="bold",fontsize=13)
plt.title(r"Heatmap - Mean Point Speed "+"\n"+r"(over hours of the day) - All road types",fontsize=13,fontweight="bold")

In [ ]:
df_corr=df_merged.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
# mask
mask = np.triu(np.ones_like(df_corr, dtype=np.bool))
# adjust mask and df
mask = mask[1:, :-1]
corr = df_corr.iloc[1:,:-1].copy()
# color map
# cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True)
# plot heatmap

# sns.heatmap(df_merged.corr(), annot=False,fmt=".1f", 
#            linewidths=0.5, cmap="coolwarm",cbar_kws={'label': 'Mean Point Speed (km/h)'})

sns.heatmap(corr, mask=mask, annot=False, fmt=".2f", 
           linewidths=0.5, cmap="coolwarm", square=True,cbar_kws={'label': 'Correlation Coefficient'})
# ticks
yticks = [i.upper() for i in corr.index]
xticks = [i.upper() for i in corr.columns]
plt.yticks(plt.yticks()[0], labels=yticks, rotation=0)
plt.xticks(plt.xticks()[0], labels=xticks)
# title
plt.xlabel('Hour of the day',fontweight="bold",fontsize=13)
plt.ylabel('Hour of the day',fontweight="bold",fontsize=13)
plt.title(r"Heatmap - Mean Point Speed (km/h) "+"\n"+r"(over hours of the day) - All road types",fontsize=13,fontweight="bold")

#### AOS Summary

In [ ]:
import glob
import os

def count_of_vals(which_hour,hour_val):
    path=r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour'+which_hour
    all_files = glob.glob(os.path.join(path, "*.csv"))

    df_aos_summary0 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
    df_aos_summary0=df_aos_summary0.groupby(['Numberplate','Event type Rename']).size().unstack(fill_value=0)
    df_aos_summary0.drop([' Indicators = OFF', 'Brakes = OFF','Headway Warning = OFF','Lane Departure Warning = OFF', 'Left Indicator = ON','Left and Right Indicator = ON',
        'Right Indicator = ON','Headway Warning (long)','Headway Warning (medium)','Left Lane Departure Warning = ON','Brakes = ON','Right Lane Departure Warning = ON'], axis = 1, inplace = True) 
    df_aos_summary0.rename(columns={'Headway Warning (short)': 'L(III)-HW'+hour_val},inplace=True, errors='raise')
    # df_aos_summary0.rename(columns={'Brakes = ON': 'Braking Events'+hour_val,
    #                             'Headway Warning (long)': 'L(I)-HW'+hour_val,
    #                             'Headway Warning (medium)': 'L(I)-HW'+hour_val,
    #                             'Headway Warning (short)': 'L(I)-HW'+hour_val,
    #                             'Left Lane Departure Warning = ON': 'L-LDW'+hour_val,
    #                             'Right Lane Departure Warning = ON': 'R-LDW'+hour_val},inplace=True, errors='raise')
    df_aos_summary0.reset_index(inplace=True)
    # df_aos_summary0.drop(['Event type Rename'],axis=1,inplace=True)
    
    return df_aos_summary0

In [ ]:
df_aos_summary0=count_of_vals('\Hour0','0')
df_aos_summary1=count_of_vals('\Hour1','1')
df_aos_summary2=count_of_vals('\Hour2','2')
df_aos_summary3=count_of_vals('\Hour3','3')
df_aos_summary4=count_of_vals('\Hour4','4')
df_aos_summary5=count_of_vals('\Hour5','5')
df_aos_summary6=count_of_vals('\Hour6','6')
df_aos_summary7=count_of_vals('\Hour7','7')
df_aos_summary8=count_of_vals('\Hour8','8')
df_aos_summary9=count_of_vals('\Hour9','9')
df_aos_summary10=count_of_vals('\Hour10','10')
df_aos_summary11=count_of_vals('\Hour11','11')
df_aos_summary12=count_of_vals('\Hour12','12')
df_aos_summary13=count_of_vals('\Hour13','13')
df_aos_summary14=count_of_vals('\Hour14','14')
df_aos_summary15=count_of_vals('\Hour15','15')
df_aos_summary16=count_of_vals('\Hour16','16')
df_aos_summary17=count_of_vals('\Hour17','17')
df_aos_summary18=count_of_vals('\Hour18','18')
df_aos_summary19=count_of_vals('\Hour19','19')
df_aos_summary20=count_of_vals('\Hour20','20')
df_aos_summary21=count_of_vals('\Hour21','21')
df_aos_summary22=count_of_vals('\Hour22','22')
df_aos_summary23=count_of_vals('\Hour23','23')

In [ ]:
df['Month'] = pd.DatetimeIndex(df['Date']).month

In [ ]:
import pandas as pd
from functools import reduce

# compile the list of dataframes you want to merge
data_frames = [df_aos_summary0, df_aos_summary1, df_aos_summary2,df_aos_summary3,df_aos_summary4,df_aos_summary5,df_aos_summary6,df_aos_summary7,df_aos_summary8,df_aos_summary9,df_aos_summary10,df_aos_summary11,df_aos_summary12,df_aos_summary13,df_aos_summary14,
               df_aos_summary15,df_aos_summary16,df_aos_summary17,df_aos_summary18,df_aos_summary19,df_aos_summary20,df_aos_summary21,df_aos_summary22,df_aos_summary23]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Numberplate'],
                                            how='inner'), data_frames)
df_merged

In [ ]:
# df_merged.rename(columns={'Braking Events0': '0',
#                                 'Braking Events1': '1',
#                                 'Braking Events2': '2',
#                                 'Braking Events3': '3',
#                                 'Braking Events4': '4',
#                                 'Braking Events5': '5',
#                                 'Braking Events6': '6',
#                                 'Braking Events7': '7',
#                                 'Braking Events8': '8',
#                                 'Braking Events9': '9',
#                                 'Braking Events10': '10',
#                                 'Braking Events11': '11',
#                                 'Braking Events12': '12',
#                                 'Braking Events13': '13',
#                                 'Braking Events14': '14',
#                                 'Braking Events15': '15',
#                                 'Braking Events16': '16',
#                                 'Braking Events17': '17',
#                                 'Braking Events18': '18',
#                                 'Braking Events19': '19',
#                                 'Braking Events20': '20',
#                                 'Braking Events21': '21',
#                                 'Braking Events22': '22',
#                                 'Braking Events23': '23'},inplace=True, errors='raise')

In [ ]:
# df_merged.rename(columns={'R-LDW0': '0',
#                                 'R-LDW1': '1',
#                                 'R-LDW2': '2',
#                                 'R-LDW3': '3',
#                                 'R-LDW4': '4',
#                                 'R-LDW5': '5',
#                                 'R-LDW6': '6',
#                                 'R-LDW7': '7',
#                                 'R-LDW8': '8',
#                                 'R-LDW9': '9',
#                                 'R-LDW10': '10',
#                                 'R-LDW11': '11',
#                                 'R-LDW12': '12',
#                                 'R-LDW13': '13',
#                                 'R-LDW14': '14',
#                                 'R-LDW15': '15',
#                                 'R-LDW16': '16',
#                                 'R-LDW17': '17',
#                                 'R-LDW18': '18',
#                                 'R-LDW19': '19',
#                                 'R-LDW20': '20',
#                                 'R-LDW21': '21',
#                                 'R-LDW22': '22',
#                                 'R-LDW23': '23'},inplace=True, errors='raise')

In [ ]:
df_merged.rename(columns={'L(III)-HW0': '0',
                                'L(III)-HW1': '1',
                                'L(III)-HW2': '2',
                                'L(III)-HW3': '3',
                                'L(III)-HW4': '4',
                                'L(III)-HW5': '5',
                                'L(III)-HW6': '6',
                                'L(III)-HW7': '7',
                                'L(III)-HW8': '8',
                                'L(III)-HW9': '9',
                                'L(III)-HW10': '10',
                                'L(III)-HW11': '11',
                                'L(III)-HW12': '12',
                                'L(III)-HW13': '13',
                                'L(III)-HW14': '14',
                                'L(III)-HW15': '15',
                                'L(III)-HW16': '16',
                                'L(III)-HW17': '17',
                                'L(III)-HW18': '18',
                                'L(III)-HW19': '19',
                                'L(III)-HW20': '20',
                                'L(III)-HW21': '21',
                                'L(III)-HW22': '22',
                                'L(III)-HW23': '23'},inplace=True, errors='raise')

In [ ]:
df_corr=df_merged.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 10))
# mask
mask = np.triu(np.ones_like(df_corr, dtype=np.bool))
# adjust mask and df
mask = mask[1:, :-1]
corr = df_corr.iloc[1:,:-1].copy()
# color map
# cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True)
# plot heatmap

# sns.heatmap(df_merged.corr(), annot=False,fmt=".1f", 
#            linewidths=0.5, cmap="coolwarm",cbar_kws={'label': 'Mean Point Speed (km/h)'})

sns.heatmap(corr, mask=mask, annot=False, fmt=".2f", 
           linewidths=0.5, cmap="coolwarm", square=True,cbar_kws={'label': 'Correlation Coefficient'})
# ticks
yticks = [i.upper() for i in corr.index]
xticks = [i.upper() for i in corr.columns]
plt.yticks(plt.yticks()[0], labels=yticks, rotation=0)
plt.xticks(plt.xticks()[0], labels=xticks)
# title
plt.xlabel('Hour of the day',fontweight="bold",fontsize=13)
plt.ylabel('Hour of the day',fontweight="bold",fontsize=13)
plt.title(r"Heatmap : L(III)-HW "+"\n"+r"(over hours of the day) - All road types",fontsize=13,fontweight="bold")

In [ ]:
df_merged

https://towardsdatascience.com/visualizing-three-dimensional-data-heatmaps-contours-and-3d-plots-with-python-bd718d1b42b4

In [ ]:
# Create meshgrid
fig = plt.figure(figsize=(8,6))
ax = plt.subplot(111, projection='3d')
X, Y = np.meshgrid(np.linspace(0, 24,1), np.linspace(0, 24, 1))

In [ ]:
plot = ax.plot_surface(X=X, Y=Y, Z=df_merged.corr(), cmap='YlGnBu_r', vmin=0, vmax=200)
plot

AOS Summary count plot

In [ ]:

%%capture
import pandas as pd
# import modin.pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
# import vaex # https://vaex.io/docs/index.html"
import pathlib
from pathlib import *
import os
import pickle
# import cufflinks as cf
# import chart_studio.plotly as py
import seaborn as sns
import plotly.express as px
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot

init_notebook_mode(connected=True)
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [ ]:
import glob
import os

path=r'D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour\Hour23'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_aos_hour0 = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
# df_TD_hour0 = df_TD_hour0.drop(df_TD_hour0[df_TD_hour0['Point_speed']==0].index)
# df_TD_hour0.drop(['Unnamed: 0', 'Numberplate','TNO_Time-stamp','TNO_Time-stamp_hour'], axis = 1, inplace = True) 
# df_TD_hour0.rename(columns={'Point_speed':'0'},
#           inplace=True, errors='raise')
# # df_TD_hour3["Hour"] = df_TD_hour3["Hour"].astype(int)
# df_aos_hour0['Count'] = np.arange(len(df_aos_hour0))

df_aos_hour0


In [ ]:
def total_aos_event_count(df):
    df=df.groupby(['Event type Rename','Road_type_Rename']).size().unstack(fill_value=0)
    df.reset_index(inplace=True)
    return df

In [ ]:
df_aos_hour0=total_aos_event_count(df_aos_hour0)
df_aos_hour0

In [ ]:
df_aos_hour0.to_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour\df_aos_hour23.csv")

In [ ]:
distance_travelled=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour\Urban\df_aos_hour0.csv")
distance_travelled.columns

In [ ]:
sns.set_palette("pastel")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Brakes = ON"],linewidth=3.5,label="Braking Events")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Headway Warning (long)"],linewidth=3.5,label="HW-L(I)")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Headway Warning (medium)"],linewidth=3.5,label="HW-L(II)")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Headway Warning (short)"],linewidth=3.5,label="HW-L(III)")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Left Lane Departure Warning = ON"],linewidth=3.5,label="L-LDW")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Right Lane Departure Warning = ON"],linewidth=3.5,label="R-LDW")
plt.xlabel("Hour of the day")
plt.ylabel("Number of Data Points")
plt.legend(title="AOS Events")
plt.title("AOS events recorded in Urban areas")

In [ ]:
distance_travelled1=pd.read_csv(r"D:\AOS FOT\Octo\CSV Export\AOS_SUMMARY\split_data\Hour\Motorway\df_aos_hour0.csv")
distance_travelled1.columns

In [ ]:
# sns.set_palette("plasma")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Brakes = ON"],linewidth=3.5,label="Braking Events")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Headway Warning (long)"],linewidth=3.5,label="HW-L(I)")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Headway Warning (medium)"],linewidth=3.5,label="HW-L(II)")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Headway Warning (short)"],linewidth=3.5,label="HW-L(III)")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Left Lane Departure Warning = ON"],linewidth=3.5,label="L-LDW")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Right Lane Departure Warning = ON"],linewidth=3.5,label="R-LDW")
plt.xlabel("Hour of the day")
plt.ylabel("Number of Data Points")
plt.legend(title="AOS Events")
plt.title("AOS events recorded in Urban areas")

In [ ]:
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Left Lane Departure Warning = ON"],linewidth=3.5,label="Urban")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Left Lane Departure Warning = ON"],linewidth=3.5,label="Motorway")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Left Lane Departure Warning = ON"],linewidth=3.5,label="Motorway")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Left Lane Departure Warning = ON"],linewidth=3.5,label="Urban")

In [ ]:
distance_travelled.columns

In [ ]:
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Right Lane Departure Warning = ON"],linewidth=3.5,label="R-LDW")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Right Indicator = ON"],linewidth=3.5,label="Right Indicator = ON")
plt.ylabel("Number of Data Points")
plt.title("AOS Events recorded in Urban Areas")
# sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Right Lane Departure Warning = ON"],linewidth=3.5,label="Motorway")

In [ ]:
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Right Lane Departure Warning = ON"],linewidth=3.5,label="R-LDW")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Right Indicator = ON"],linewidth=3.5,label="Right Indicator = ON")
plt.ylabel("Number of Data Points")
plt.title("AOS Events recorded on Motorways")

In [ ]:
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Left Lane Departure Warning = ON"],linewidth=3.5,label="L-LDW")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Left Indicator = ON"],linewidth=3.5,label="Left Indicator = ON")
plt.ylabel("Number of Data Points")
plt.title("AOS Events recorded on Motorways")

In [ ]:
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Left Lane Departure Warning = ON"],linewidth=3.5,label="L-LDW")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Left Indicator = ON"],linewidth=3.5,label="Left Indicator = ON")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Right Lane Departure Warning = ON"],linewidth=3.5,label="R-LDW")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Right Indicator = ON"],linewidth=3.5,label="Right Indicator = ON")
plt.ylabel("Number of Data Points")
plt.title("AOS Events recorded on Motorways")
plt.legend(loc="upper right")

In [ ]:
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Left Lane Departure Warning = ON"],linewidth=3.5,label="L-LDW")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Left Indicator = ON"],linewidth=3.5,label="Left Indicator = ON")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Right Lane Departure Warning = ON"],linewidth=3.5,label="R-LDW")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Right Indicator = ON"],linewidth=3.5,label="Right Indicator = ON")
plt.ylabel("Number of Data Points")
plt.title("AOS Events recorded in Urban Areas")


In [ ]:
distance_travelled.columns

In [ ]:
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Headway Warning (long)"],linewidth=3.5,label="HW-L(I)")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Headway Warning (medium)"],linewidth=3.5,label="HW-L(II)")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Headway Warning (short)"],linewidth=3.5,label="HW-L(III)")
sns.lineplot(x=distance_travelled["Hour"],y=distance_travelled["Brakes = ON"],linewidth=3.5,label="Braking Events")
plt.ylabel("Number of Data Points")
plt.title("AOS Events recorded in Urban Areas")

In [ ]:
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Headway Warning (long)"],linewidth=3.5,label="HW-L(I)")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Headway Warning (medium)"],linewidth=3.5,label="HW-L(II)")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Headway Warning (short)"],linewidth=3.5,label="HW-L(III)")
sns.lineplot(x=distance_travelled1["Hour"],y=distance_travelled1["Brakes = ON"],linewidth=3.5,label="Braking Events")
plt.ylabel("Number of Data Points")
plt.title("AOS Events recorded on Motorways")